# New dataset for abundance analysis

## Things to do
* Separate samples into insulin/basal
* For abundance cut-offs, averages the metabolite abundance across samples in the same group
* For DE list, use paired t-tests to determine DE metabolites. Beware of the zeros.
* Check how long the list is

## class definition of a sample

In [ ]:
class MetabolomicsSample:
    def __init__(self):
        '''
        id, liver fat, insulin infusion, serum type, patient, compound:abundance
        '''

## MTBLS298
242 Unique Chebi IDs

In [1]:
directory = '/home/zxu/Documents/mscbioinfo/data/MTBLS298/'
maf = directory + 'm_catheterization_study_metabolite_profiling_mass_spectrometry_v2_maf.tsv'

In [2]:
db_id = []
with open(maf, 'r') as fh:
    for line in fh.readlines():
        fields = line.rstrip().split('\t')
        #print(fields[0], fields[4], fields[11], fields[14], fields[16], fields[17], fields[21:81])# 0 4 11 14 16 17 21-80
        db_id.append(fields[0][1:-1])

In [3]:
chebi_id = []
for chem in db_id:
    if chem.startswith('CHEBI'):
        chebi_id.append(chem)

In [4]:
set(chebi_id)

{'CHEBI:111506',
 'CHEBI:111507',
 'CHEBI:111508',
 'CHEBI:111510',
 'CHEBI:111512',
 'CHEBI:111514',
 'CHEBI:111517',
 'CHEBI:111518',
 'CHEBI:111520',
 'CHEBI:1148',
 'CHEBI:131185',
 'CHEBI:131187',
 'CHEBI:131192',
 'CHEBI:131193',
 'CHEBI:131195',
 'CHEBI:131197',
 'CHEBI:131198',
 'CHEBI:131199',
 'CHEBI:131328',
 'CHEBI:131329',
 'CHEBI:131330',
 'CHEBI:131332',
 'CHEBI:131333',
 'CHEBI:131334',
 'CHEBI:131335',
 'CHEBI:131336',
 'CHEBI:131339',
 'CHEBI:131340',
 'CHEBI:131341',
 'CHEBI:131342',
 'CHEBI:131343',
 'CHEBI:131344',
 'CHEBI:131346',
 'CHEBI:131348',
 'CHEBI:131350',
 'CHEBI:131352',
 'CHEBI:131353',
 'CHEBI:131354',
 'CHEBI:131356',
 'CHEBI:131357',
 'CHEBI:131360',
 'CHEBI:131362',
 'CHEBI:131363',
 'CHEBI:131364',
 'CHEBI:131367',
 'CHEBI:131368',
 'CHEBI:131369',
 'CHEBI:131370',
 'CHEBI:131373',
 'CHEBI:131374',
 'CHEBI:131375',
 'CHEBI:131376',
 'CHEBI:131377',
 'CHEBI:131379',
 'CHEBI:131380',
 'CHEBI:131381',
 'CHEBI:131382',
 'CHEBI:131384',
 'CHEBI:131385',

## MTBLS358
m_SHOT_mass_spectrometry_v4.maf 100

m_TAG_mass_spectrometry_v4.maf 29

m_CER_mass_spectrometry_v4.maf 45

m_EICO_mass_spectrometry_v4.maf 17


In [76]:
directory = '/home/zxu/Documents/mscbioinfo/data/MTBLS358/'
maf = directory + 'm_EICO_mass_spectrometry_v4.maf'

In [77]:
db_id = []
with open(maf, 'r') as fh:
    for line in fh.readlines():
        fields = line.rstrip().split('\t')
        #print(fields[0], fields[4], fields[11], fields[14], fields[16], fields[17], fields[21:81])# 0 4 11 14 16 17 21-80
        db_id.append(fields[0][1:-1])

In [78]:
chebi_id = []
for chem in db_id:
    if chem.startswith('CHEBI'):
        chebi_id.append(chem)

In [79]:
len(set(chebi_id))

17

## Conversion: Chebi to KEGG

In [5]:
from bioservices import KEGG, ChEBI

In [6]:
# Not used anymore
#k = KEGG(verbose=False)
#map_chebi_kegg = k.conv("compound", "chebi")

In [7]:
ch = ChEBI()

In [8]:
def conv_chebi_kegg(chebi_id, chebi_instance):
    '''
    converting chebi ids to kegg ids
    if the chebi entry does not have database links
    return the ascii name of that entry
    '''
    res = chebi_instance.getCompleteEntity(chebi_id)
    try:
        for link in res.DatabaseLinks:
            if link.type == 'KEGG COMPOUND accession':
                kegg_id = link.data
                return kegg_id
    except AttributeError:
        pass
    return res.chebiAsciiName

In [10]:
converted_kegg_ids = []
for compound in chebi_id:
    cpdid = conv_chebi_kegg(compound, ch)
    converted_kegg_ids.append(cpdid)
    #print(cpdid)

In [31]:
for xdxd in set(converted_kegg_ids):
    print(xdxd)

2-amino-4-(3-cyclohexen-1-yl)-7,7-dimethyl-5-oxo-5,6,7,8-tetrahydrochromene-3-carbonitrile
3,5-dihydroxyphenylacetic acid
2-ethyl-3-ketopentanoic acid
C00184
C01607
N-methylleucine
t-butyl hexadecanoate
2-[2-(2-methoxyethoxy)ethoxy]-1,3-dioxolane
1,2,3-butanetriol
C00379
dimethylmalonic acid
(3Z,6E)-3,6-bis(methoxyimino)hexane-1,2,4,5-tetrol
10-undecen-1-ol
(2-ethoxyethoxy)acetic acid
C00090
C02341
2-[(9E)-9-octadecenoyl]glycerol
C16439
C00116
C05587
citrulline
2-(3-hydroxyphenyl)propionic acid
1-monostearoylglycerol
C10164
methyl (2E)-3-(3-hydroxyphenyl)acrylate
1-monomyristoylglycerol
(Z)-1,2-ethenediol
methyl cyclopropanecarboxylate
C16438
(2E)-2-(hydroxyimino)isohexanoic acid
2-palmitoylglycerol
C01733
arabinitol
ethyl L-methioninate
tert-butanol
2-[(9E,12E,15E)-octadecatrienoyl]glycerol
1-isopentyl-2,3,4,6-tetramethylbenzene
C06772
C00716
2-methylbutanedioic acid
C00383
C00303
C01571
C02679
2,3-dihydroxybutanoic acid
C00806
C00158
C00805
2-hydroxy-2-methylbutyric acid
5-hydroxyhex

In [29]:
count = 0
for keg_id in set(converted_kegg_ids):
    if keg_id.startswith('C') and len(keg_id) == 6:
        count += 1
print(count)

108
